# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [24]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
%pip install yfinance

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

Note: you may need to restart the kernel to use updated packages.
ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [25]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = "_".join([f"{k}-{v}" for k, v in meta.items()]) if meta else ""
    fname = f"{prefix}_{mid}_{ts()}.csv" if mid else f"{prefix}_{ts()}.csv"
    path = RAW / fname
    
    df.to_csv(path, index=False)
    print("Saved", path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    result = {
        "missing": missing,
        "shape": df.shape,
        "na_total": int(df.isna().sum().sum())
    }
    print("Validation:", result)
    return result

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [27]:
SYMBOL = "AAPL"

key_env = os.getenv("ALPHAVANTAGE_API_KEY")
df_api = None
used_alpha = False

if key_env:
    try:
        r = requests.get(
            "https://www.alphavantage.co/query",
            params={"function":"TIME_SERIES_DAILY_ADJUSTED","symbol":SYMBOL,"outputsize":"compact","apikey":key_env},
            timeout=30
        )
        r.raise_for_status()
        js = r.json()
        if any(k in js for k in ("Error Message","Note","Information","message")):
            raise RuntimeError(js.get("Error Message") or js.get("Note") or js.get("Information") or js.get("message"))
        ts_key = next(k for k in js.keys() if "Time Series" in k)
        tmp = pd.DataFrame(js[ts_key]).T.reset_index().rename(columns={"index":"date"})
        close_col = "5. adjusted close" if "5. adjusted close" in tmp.columns else "4. close"
        df_api = tmp[["date", close_col]].rename(columns={close_col:"adj_close"})
        used_alpha = True
    except Exception as e:
        print("alpha_vantage_fallback:", e)

if df_api is None:
    import yfinance as yf
    df_yf = yf.download(SYMBOL, period="3mo", interval="1d", progress=False, auto_adjust=False).reset_index()
    if isinstance(df_yf.columns, pd.MultiIndex):
        df_yf.columns = ["_".join([str(p) for p in col if p != ""]).strip("_") for col in df_yf.columns.to_list()]
    cand = [c for c in df_yf.columns if str(c).lower() in ("adj close","close","adj_close","close_"+SYMBOL.lower(),"adj close_"+SYMBOL.lower())]
    if not cand:
        cand = [c for c in df_yf.columns if "adj" in str(c).lower() and "close" in str(c).lower()] or [c for c in df_yf.columns if str(c).lower()=="close"]
    close_col = cand[0]
    date_col = "Date" if "Date" in df_yf.columns else [c for c in df_yf.columns if str(c).lower()=="date"][0]
    df_api = df_yf[[date_col, close_col]].rename(columns={date_col:"date", close_col:"adj_close"})

s = pd.Series(df_api["adj_close"]).astype("string")
df_api["adj_close"] = pd.to_numeric(s, errors="coerce")
df_api["date"] = pd.to_datetime(df_api["date"], errors="coerce")
df_api = df_api.dropna(subset=["date","adj_close"]).sort_values("date")

v_api = validate(df_api, ["date","adj_close"]); v_api
_ = save_csv(df_api, prefix="api", source="alpha" if used_alpha else "yfinance", symbol=SYMBOL)

alpha_vantage_fallback: Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints
Validation: {'missing': [], 'shape': (63, 2), 'na_total': 0}
Saved data\raw\api_source-yfinance_symbol-AAPL_20250817-225207.csv


In [28]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data\raw\api_source-alpha_symbol-AAPL_20250817-225209.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [29]:
SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "AFE-Homework/1.0"}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

Validation: {'missing': [], 'shape': (880, 8), 'na_total': 758}


{'missing': [], 'shape': (880, 8), 'na_total': 758}

In [30]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data\raw\scrape_site-example_table-markets_20250817-225257.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.